In [ ]:
from mpcontribs.client import Client
from pandas import read_excel, MultiIndex
from pymatgen.ext.matproj import MPRester
from unflatten import unflatten

In [ ]:
name = 'defect_genome_pcfc_materials'
client = Client()
mpr = MPRester()

**Retrieve and update project info**

In [ ]:
client.get_project(name).display()

In [ ]:
client.projects.update_entry(pk=name, project={
    "references[0]": {"label": "ACS", "url": "https://doi.org/10.1021/acs.jpcc.7b08716"}
}).result()

**Prepare contributions**

In [ ]:
df = read_excel('/Users/patrick/gitrepos/mp/MPContribs/mpcontribs-data/DefectGenome_JPCC-data_MP.xlsx')
df.columns = MultiIndex.from_arrays([
    ['', '', '', 'Eᶠ', 'Eᶠ', 'Eᶠ', 'Eᶠ', 'ΔEᵢ'],
    ['A', 'B', 'a', 'ABO₃', 'Yᴮ', 'Vᴼ', 'Hᵢ', 'Yᴮ−Hᵢ']
])
units = {'A': '', 'B': '', 'a': 'Å'}
df

In [ ]:
contributions = []
for idx, row in df.iterrows():
    A, B = row[df.columns[0]], row[df.columns[1]]
    formula = f'{A}{B}O3'
    data = mpr.get_data(formula, prop="volume")

    if len(data) > 1:
        volume = row[df.columns[2]]**3
        for d in data:
            d['dV'] = abs(d['volume']-volume)
        data = sorted(data, key=lambda item: item['dV'])
    elif not data:
        print(formula, 'not found on MP')
        continue

    identifier = data[0]['material_id']
    #print(idx, formula, identifier)
    
    data = {}
    for col in df.columns:
        flat_col = ".".join([c for c in col if c])
        unit = units.get(flat_col, 'eV')
        data[flat_col] = f'{row[col]} {unit}' if unit else row[col]

    contrib = {
        'project': name, 'identifier': identifier, 'is_public': True,
        'data': unflatten(data)
    }
    contributions.append(contrib)
    
len(contributions)

**Submit contributions**

In [ ]:
client.delete_contributions(name)
client.submit_contributions(contributions)

**Query contributions**

In [ ]:
query = {
    "project": name,
#     "formula__contains": "Mg",
    "data__A__contains": "Mg",
    "data__a__value__lte": 4.1,
    "data__Eᶠ__ABO₃__value__lte": 3.2,
    "_order_by": "data__a__value",
    "order": "desc",
    "_fields": [
        "id", "identifier", "formula",
        "data.A", "data.a.value", "data.Eᶠ.ABO₃.value"
    ]   
}
client.contributions.get_entries(**query).result()